In [ ]:
import seaborn as sns
import pandas as pd
import pyspark.pandas as ps
import mysql.connector
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import matplotlib.pyplot as plt

In [ ]:
db = mysql.connector.connect(
    host = "<your_host>",
    user = "<your_user" , 
    password = "<your_password>", 
    database = "Big_Data"

In [ ]:
print(db)

In [ ]:
cursor = db.cursor()


In [ ]:
sql_query1 ='''select dimcustomer.YearlyIncome,dimgeography.StateProvinceName ,
 dimgeography.EnglishCountryRegionName , factinternetsales.TotalProductCost
 from dimcustomer 
join dimgeography on (dimcustomer.GeographyKey = dimgeography.GeographyKey)
join factinternetsales on (dimcustomer.CustomerKey = factinternetsales.CustomerKey);'''

In [ ]:
cursor.execute(sql_query1)


In [ ]:
data1= cursor.fetchall()

In [ ]:
for row in data1 : print(row)

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
customer_df = ps.DataFrame (data1 )

In [ ]:
new_column_names = ['YearlyIncome','StateProvinceName','EnglishCountryRegionName', 'TotalProductCost']
customer_df.columns = new_column_names

In [ ]:
customer_df.head()

In [ ]:
pandas_df = customer_df.to_pandas()
pandas_df = pandas_df.rename(columns={'TotalProductCost': 'TotalSales'}, inplace=False)
pandas_df = pandas_df.rename(columns={'EnglishCountryRegionName': 'Region'}, inplace=False)
pandas_df.head()

In [ ]:
pandas_df.columns


In [ ]:
pandas_df.describe()

In [ ]:
pandas_df.isnull().sum()

In [ ]:
group_df_by_total = pandas_df.groupby('Region').sum('TotalSales')
group_df_by_total = group_df_by_total.drop('YearlyIncome', axis=1)
group_df_by_total


In [ ]:

barplot = plt.bar(group_df_by_total.index, group_df_by_total['TotalSales'] )
plt.title('TOTAL SALES BY REGION')
plt.xlabel('Region')
plt.ylabel('Total Sales ')
plt.bar_label(barplot,label_type = 'edge', padding = 2 )

plt.show()

In [ ]:
group_df_by_total.max()

In [ ]:
group_df_by_income = pandas_df.groupby('Region').mean('YearlyIncome' )
group_df_by_income = group_df_by_income.drop('TotalSales', axis=1)
group_df_by_income

In [ ]:
group_df_by_income.max()

In [ ]:
barplot2 = plt.bar(group_df_by_income.index, group_df_by_income['YearlyIncome'] )
plt.title('CITIZENS YEARLY INCOME BY REGION ')
plt.xlabel('Region')
plt.ylabel('Average Yearly Income ')
plt.bar_label(barplot2,label_type = 'edge', padding = 2 )
plt.show()

In [ ]:
merged_df = pd.merge(group_df_by_income,group_df_by_total , on ='Region')
merged_df

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='YearlyIncome', y='TotalSales', hue='Region', palette='viridis', s=100)
sns.regplot(data=merged_df, x='YearlyIncome', y='TotalSales', scatter=False)
plt.xlabel('Citizes Yearly Income by Region')
plt.ylabel('Total Sales by Region')
plt.title('TOTAL SALES US YEARLY INCOME')


plt.show()

In [ ]:
df_encoded = pd.get_dummies(pandas_df, columns=['StateProvinceName'] ,dtype=float)
df_encoded

In [ ]:
from sklearn.cluster import KMeans

features_for_kmeans = df_encoded[['YearlyIncome', 'TotalSales']]

kmeans = KMeans(n_clusters=3, n_init='auto', random_state=24)
df_encoded['Cluster'] = kmeans.fit_predict(features_for_kmeans)



kmeans.cluster_centers_

In [ ]:
inertia = []

for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    kmeans.fit(features_for_kmeans)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

features = df_encoded.drop(['Region'], axis=1)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components= 10)
principal_components = pca.fit_transform(scaled_features)

In [ ]:

principal_components_df = pd.DataFrame(data=principal_components, columns=[f'PC{i}' for i in range(1, 10+1)])


explained_variance_ratio = pca.explained_variance_ratio_
total_explained_variance = explained_variance_ratio.sum()
total_explained_variance


In [ ]:

plt.figure(figsize=(10, 6))
scatter = plt.scatter(principal_components_df['PC1'], principal_components_df['PC2'], c=df_encoded['Cluster'], cmap='viridis', alpha=0.5)
plt.title('Scatter Plot of Principal Components with Clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.show()